In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import pdist, squareform

from sklearn.metrics import pairwise_distances 
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.manifold import MDS
from sklearn.manifold import TSNE
from sklearn import metrics
from sklearn.metrics import silhouette_score

import scikitplot as skplt

import random
random.seed(820)

In [ ]:
#distance to measure the difference between animes
df = pd.read_csv('anime_new.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head(3)

df_num = df.select_dtypes('number')
df_num.info()

df_num = df_num.dropna()
df_num.isnull().sum()

df_num.head(3)
df_num.describe

#22 columns
df_num.info()


ds = StandardScaler().fit_transform(df_num)
data_scaled = pd.DataFrame(ds,columns=df_num.columns)
data_scaled.describe().T

pca = PCA(0.95)
pcs = pca.fit_transform(ds)
varexp = pca.explained_variance_ratio_
pca.n_components_

plt.title("Cumulative Explained Variance")
plt.plot(range(1, len(varexp)+1), np.cumsum(varexp))
plt.axhline(.95)
plt.show()

columns_name = []
for i in range(pca.n_components_):
    columns_name.append('pca_'+ str(i+1))
pca_df = pd.DataFrame(pcs,columns=columns_name)

krange_pca = range(2,20)
inertia_pca =[]
silo_pca = []
for k in krange_pca:
    kmodel = KMeans(k)
    k_labs = kmodel.fit_predict(pca_df)
    inertia_pca.append(kmodel.inertia_)
    silo_pca.append(silhouette_score(pca_df,k_labs))

#Show Inertia plot
sns.lineplot(krange_pca,inertia_pca)
plt.title('k value and inertia')
plt.show()

#Show Silhouette plot
sns.lineplot(krange_pca, silo_pca)
plt.title('k value and silhouette score')
plt.show()

k4 = KMeans(4)
k4_labs = k4.fit_predict(pca_df)

skplt.metrics.plot_silhouette(pca_df, k4_labs, title="Kmeans-3", figsize=(15,5))
plt.show()


df_num['k4_labs'] = k4_labs
df_num['k4_labs'].value_counts(sort=False)

profile = df_num.groupby('k4_labs').mean().T
profile

plt.figure(figsize=(12, 6))
#pal = sns.color_palette("blue")
sns.heatmap(profile, center=0, xticklabels=profile.columns)
plt.title('Characteristics of 4 customer clusters')
plt.show()